In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
import random
import time 
import cProfile
import cython
from sklearn.preprocessing import normalize
from multiprocessing import Pool, cpu_count

In [2]:
K = 5

file_path = "lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv"

# Load interaction log
interactions = pd.read_csv(file_path, sep="\t", header=None,
                           names=["user_id", "timestamp", "artist_id", "artist_name", "track_id", "track_name"], on_bad_lines='skip')

In [3]:
# Count number of times each user listened to each artist
playcounts = interactions.groupby(["user_id", "artist_name"]).size().reset_index(name="playcount")

# Create mappings: user/artist to index
user_id_to_idx = {user: idx for idx, user in enumerate(playcounts["user_id"].unique())}
artist_name_to_idx = {artist: idx for idx, artist in enumerate(playcounts["artist_name"].unique())}

# Map to index
playcounts["user_idx"] = playcounts["user_id"].map(user_id_to_idx)
playcounts["artist_idx"] = playcounts["artist_name"].map(artist_name_to_idx)

# Build the sparse matrix (UAM)
UAM = csr_matrix((playcounts["playcount"],
                  (playcounts["user_idx"], playcounts["artist_idx"])),
                 shape=(len(user_id_to_idx), len(artist_name_to_idx)))

# normalize rows (for cosine similarity)

UAM = normalize(UAM, norm='l2', axis=1)

# Create lists for reverse lookup
user_ids = np.array(list(user_id_to_idx.keys()))
artist_ids = np.array(list(artist_name_to_idx.keys()))


if __name__ == '__main__':
    start_time = time.time()
    for u in range(0, UAM.shape[0]):
        print("Seed user-id: " + str(user_ids[u]))

        # Get normalized playcount vector for current user
        pc_vec = UAM.getrow(u)

        # Compute cosine similarity between this user and all users
        uU_sim = pc_vec.dot(UAM.transpose()).tocoo()
        uU_user_idx = uU_sim.col
        uU_data = uU_sim.data

        # Remove self-similarity
        uU_data[uU_user_idx == u] = 0

        # Eliminate zeros
        uU_sim.data = uU_data
        uU_sim = uU_sim.tocsr()
        uU_sim.eliminate_zeros()
        uU_sim = uU_sim.tocoo()

        # Re-assign user indices and scores
        uU_user_idx = uU_sim.col
        uU_data = uU_sim.data

        # Sort by similarity score
        sort_index = np.argsort(uU_data)

        # Select top-K nearest neighbors
        # Note that uU_user_idx indeed provides the indices for users in UAM
        recommended_user_idx = uU_user_idx[sort_index[-K:]]
        # Get user_ids corresponding to nearest neighbors
        recommended_user_ids = user_ids[recommended_user_idx]
        # Get similarity score for nearest neighbors
        recommended_user_scores = uU_data[sort_index[-K:]]

        print("Nearest K=" + str(K) + " neighbors' user-ids: ", recommended_user_ids.flatten())

        # Get all artists these similar users have listened to
        recommended_artists_idx = []
        for u_idx in recommended_user_idx:
            recommended_artists_idx.extend(list(UAM.getrow(u_idx).indices))

        # Remove duplicates and sort
        recommended_artists_idx = sorted(set(recommended_artists_idx))

        # Remove artists already known to seed user
        recommended_artists_idx = np.setdiff1d(recommended_artists_idx, pc_vec.indices)

        # Narrow down to random 5 artist out of all artists that similar users listen to for illustration purpose
        random_indices = random.sample(range(len(recommended_artists_idx)), 5)
        random_elements = [recommended_artists_idx[i] for i in random_indices]

        print("Indices of " + str(len(random_elements)) + " recommended artists: ", random_elements)
        print("Recommended artist names:", [artist_ids[i] for i in random_elements])
        print('-' * 80)

    end_time = time.time()
    total_users = UAM.shape[0]
    avg_time = (end_time - start_time) / total_users
    print(f"Average time per user recommendation: {avg_time:.4f} seconds")

Seed user-id: user_000001
Nearest K=5 neighbors' user-ids:  ['user_000168' 'user_000844' 'user_000862' 'user_000629' 'user_000074']
Indices of 5 recommended artists:  [np.int32(17530), np.int32(6951), np.int32(8550), np.int32(14092), np.int32(34080)]
Recommended artist names: [np.str_('Superchunk'), np.str_('Kansas'), np.str_('George Shearing'), np.str_('Kukl'), np.str_('Aguayo/Rossknecht')]
--------------------------------------------------------------------------------
Seed user-id: user_000002
Nearest K=5 neighbors' user-ids:  ['user_000513' 'user_000726' 'user_000143' 'user_000238' 'user_000673']
Indices of 5 recommended artists:  [np.int32(2180), np.int32(3247), np.int32(12868), np.int32(10678), np.int32(2535)]
Recommended artist names: [np.str_('Funeral For A Friend'), np.str_('Jay-Z'), np.str_('Missy Elliott'), np.str_('Hundred Reasons'), np.str_('Ten Years After')]
--------------------------------------------------------------------------------
Seed user-id: user_000003
Nearest

In [4]:
# Count how many times each user listened to each artist
playcounts = interactions.groupby(["user_id", "artist_name"]).size().reset_index(name="playcount")

# Create mapping dictionaries for user/artist to index
user_id_to_idx = {user: idx for idx, user in enumerate(playcounts["user_id"].unique())}
artist_name_to_idx = {artist: idx for idx, artist in enumerate(playcounts["artist_name"].unique())}

# Map user/artist names to indices in the dataframe
playcounts["user_idx"] = playcounts["user_id"].map(user_id_to_idx)
playcounts["artist_idx"] = playcounts["artist_name"].map(artist_name_to_idx)

# Build the user-artist matrix (UAM) in sparse format
UAM = csr_matrix((playcounts["playcount"],
                  (playcounts["user_idx"], playcounts["artist_idx"])),
                 shape=(len(user_id_to_idx), len(artist_name_to_idx)))

# Normalize rows (each row becomes a unit vector for cosine similarity)
UAM = normalize(UAM, norm='l2', axis=1)

# Precompute UAM transpose for efficient cosine similarity calculations
# This avoids repeatedly computing UAM.transpose() within the loop.
UAM_T = UAM.transpose().tocsr()

# Create lists for reverse lookup
user_ids = np.array(list(user_id_to_idx.keys()))
artist_ids = np.array(list(artist_name_to_idx.keys()))

if __name__ == '__main__':
    start_time = time.time()
    num_users = UAM.shape[0]
    
    for u in range(num_users):
        print("Seed user-id:", user_ids[u])
        
        # Retrieve the normalized playcount vector (row) for the current user
        pc_vec = UAM.getrow(u)
        
        # Compute cosine similarity between the seed user and all other users
        # Using the precomputed UAM_T speeds up the dot-product calculation.
        sims = pc_vec.dot(UAM_T)
        
        # Convert the similarity result to COO format to easily access indices and data.
        sims_coo = sims.tocoo()
        
        # Remove self-similarity by filtering out the element where the column index equals the seed user.
        mask = sims_coo.col != u
        neighbor_indices = sims_coo.col[mask]
        neighbor_scores = sims_coo.data[mask]
        
        # Proceed only if there is at least one similar user
        if neighbor_scores.size == 0:
            print("No similar users found for", user_ids[u])
            print('-' * 80)
            continue
        
        # Use argpartition to quickly find indices of top K similarities.
        # For cases where there are fewer than K neighbors, return all.
        if neighbor_scores.size > K:
            top_k_idx = np.argpartition(neighbor_scores, -K)[-K:]
            # Sort the top K neighbors in descending order by their similarity scores.
            sorted_order = np.argsort(neighbor_scores[top_k_idx])[::-1]
            top_neighbors = neighbor_indices[top_k_idx][sorted_order]
        else:
            sorted_order = np.argsort(neighbor_scores)[::-1]
            top_neighbors = neighbor_indices[sorted_order]
        
        # Retrieve the actual user IDs for the top K neighbors
        recommended_user_ids = user_ids[top_neighbors]
        print(f"Nearest K={K} neighbors' user-ids:", recommended_user_ids.flatten())
        
        # Get all artist indices listened to by the top K similar users in one vectorized operation.
        # This avoids looping over each neighbor.
        neighbor_rows = UAM[top_neighbors, :]
        neighbor_coo = neighbor_rows.tocoo()
        recommended_artists_idx = np.unique(neighbor_coo.col)
        
        # Remove artists already known to the seed user
        seed_artists = pc_vec.indices
        recommended_artists_idx = np.setdiff1d(recommended_artists_idx, seed_artists)
        
        # Randomly choose 5 artists (or fewer, if not enough exist) for recommendation
        if recommended_artists_idx.size > 0:
            num_to_select = min(5, recommended_artists_idx.size)
            random_elements = random.sample(list(recommended_artists_idx), num_to_select)
            print(f"Indices of {len(random_elements)} recommended artists:", random_elements)
            rec_artist_names = [artist_ids[i] for i in random_elements]
            print("Recommended artist names:", rec_artist_names)
        else:
            print("No new recommended artists found.")
            
        print('-' * 80)
    
    end_time = time.time()
    avg_time = (end_time - start_time) / num_users
    print(f"Average time per user recommendation: {avg_time:.4f} seconds")

Seed user-id: user_000001
Nearest K=5 neighbors' user-ids: ['user_000074' 'user_000629' 'user_000862' 'user_000844' 'user_000168']
Indices of 5 recommended artists: [np.int32(33819), np.int32(11275), np.int32(35833), np.int32(85196), np.int32(34275)]
Recommended artist names: [np.str_('The Amalgamation Of Soundz'), np.str_('Beth Gibbons'), np.str_('Łona'), np.str_('Sweep - Favourite Song (Remixed)'), np.str_('Ginormous')]
--------------------------------------------------------------------------------
Seed user-id: user_000002
Nearest K=5 neighbors' user-ids: ['user_000673' 'user_000238' 'user_000143' 'user_000726' 'user_000513']
Indices of 5 recommended artists: [np.int32(10598), np.int32(140032), np.int32(3330), np.int32(17998), np.int32(8698)]
Recommended artist names: [np.str_('Eddy Grant'), np.str_('Hurry Up'), np.str_('Leonard Cohen'), np.str_('Caifanes'), np.str_("Jane'S Addiction")]
--------------------------------------------------------------------------------
Seed user-id: u

In [5]:
#Thread
from concurrent.futures import ThreadPoolExecutor, as_completed
MAX_WORKERS = 8
# Build User-Artist Matrix (UAM)
playcounts = interactions.groupby(["user_id", "artist_name"]).size().reset_index(name="playcount")

user_id_to_idx = {user: idx for idx, user in enumerate(playcounts["user_id"].unique())}
artist_name_to_idx = {artist: idx for idx, artist in enumerate(playcounts["artist_name"].unique())}

playcounts["user_idx"] = playcounts["user_id"].map(user_id_to_idx)
playcounts["artist_idx"] = playcounts["artist_name"].map(artist_name_to_idx)

UAM = csr_matrix((playcounts["playcount"],
                  (playcounts["user_idx"], playcounts["artist_idx"])),
                 shape=(len(user_id_to_idx), len(artist_name_to_idx)))

UAM = normalize(UAM, norm='l2', axis=1)

user_ids = np.array(list(user_id_to_idx.keys()))
artist_ids = np.array(list(artist_name_to_idx.keys()))

UAM_T = UAM.transpose().tocsr()

num_users = UAM.shape[0]

# --- Define function to process a single user ---
def process_single_user(u):
    pc_vec = UAM.getrow(u)
    sims = pc_vec.dot(UAM_T)
    sims_coo = sims.tocoo()
    
    mask = sims_coo.col != u
    neighbor_indices = sims_coo.col[mask]
    neighbor_scores = sims_coo.data[mask]
    
    if neighbor_scores.size == 0:
        return (user_ids[u], None, None)

    if neighbor_scores.size > K:
        top_k_idx = np.argpartition(neighbor_scores, -K)[-K:]
        sorted_order = np.argsort(neighbor_scores[top_k_idx])[::-1]
        top_neighbors = neighbor_indices[top_k_idx][sorted_order]
    else:
        sorted_order = np.argsort(neighbor_scores)[::-1]
        top_neighbors = neighbor_indices[sorted_order]

    recommended_user_ids = user_ids[top_neighbors]

    neighbor_rows = UAM[top_neighbors, :]
    neighbor_coo = neighbor_rows.tocoo()
    recommended_artists_idx = np.unique(neighbor_coo.col)

    seed_artists = UAM.getrow(u).indices
    recommended_artists_idx = np.setdiff1d(recommended_artists_idx, seed_artists)

    if recommended_artists_idx.size > 0:
        num_to_select = min(5, recommended_artists_idx.size)
        random_elements = random.sample(list(recommended_artists_idx), num_to_select)
        rec_artist_names = [artist_ids[i] for i in random_elements]
        return (user_ids[u], recommended_user_ids, rec_artist_names)
    else:
        return (user_ids[u], recommended_user_ids, None)

# --- Main Program ---
if __name__ == '__main__':
    start_time = time.time()

    all_results = []

    # Use ThreadPoolExecutor to process all users in parallel
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(process_single_user, u) for u in range(num_users)]
        for future in as_completed(futures):
            res = future.result()
            all_results.append(res)

    # Print all results
    for res in all_results:
        seed_user, neighbors, rec_artists = res
        print("Seed user-id:", seed_user)
        if neighbors is None:
            print("No similar users found for", seed_user)
            print('-' * 80)
            continue
        print(f"Nearest K={K} neighbors' user-ids:", neighbors.flatten())
        if rec_artists is not None:
            print("Recommended artist names:", rec_artist_names)
        else:
            print("No new recommended artists found.")
        print('-' * 80)

    end_time = time.time()
    avg_time = (end_time - start_time) / num_users
    print(f"Average time per user recommendation: {avg_time:.4f} seconds")

Seed user-id: user_000001
Nearest K=5 neighbors' user-ids: ['user_000074' 'user_000629' 'user_000862' 'user_000844' 'user_000168']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000008
Nearest K=5 neighbors' user-ids: ['user_000864' 'user_000419' 'user_000737' 'user_000187' 'user_000123']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000002
Nearest K=5 neighbors' user-ids: ['user_000673' 'user_000238' 'user_000143' 'user_000726' 'user_000513']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kol

In [6]:
# cprofile
from concurrent.futures import ThreadPoolExecutor, as_completed
MAX_WORKERS = 8
# Build User-Artist Matrix (UAM)
playcounts = interactions.groupby(["user_id", "artist_name"]).size().reset_index(name="playcount")

user_id_to_idx = {user: idx for idx, user in enumerate(playcounts["user_id"].unique())}
artist_name_to_idx = {artist: idx for idx, artist in enumerate(playcounts["artist_name"].unique())}

playcounts["user_idx"] = playcounts["user_id"].map(user_id_to_idx)
playcounts["artist_idx"] = playcounts["artist_name"].map(artist_name_to_idx)

UAM = csr_matrix((playcounts["playcount"],
                  (playcounts["user_idx"], playcounts["artist_idx"])),
                 shape=(len(user_id_to_idx), len(artist_name_to_idx)))

UAM = normalize(UAM, norm='l2', axis=1)

user_ids = np.array(list(user_id_to_idx.keys()))
artist_ids = np.array(list(artist_name_to_idx.keys()))

UAM_T = UAM.transpose().tocsr()

num_users = UAM.shape[0]

# --- Define function to process a single user ---
def process_single_user(u):
    pc_vec = UAM.getrow(u)
    sims = pc_vec.dot(UAM_T)
    sims_coo = sims.tocoo()
    
    mask = sims_coo.col != u
    neighbor_indices = sims_coo.col[mask]
    neighbor_scores = sims_coo.data[mask]
    
    if neighbor_scores.size == 0:
        return (user_ids[u], None, None)

    if neighbor_scores.size > K:
        top_k_idx = np.argpartition(neighbor_scores, -K)[-K:]
        sorted_order = np.argsort(neighbor_scores[top_k_idx])[::-1]
        top_neighbors = neighbor_indices[top_k_idx][sorted_order]
    else:
        sorted_order = np.argsort(neighbor_scores)[::-1]
        top_neighbors = neighbor_indices[sorted_order]

    recommended_user_ids = user_ids[top_neighbors]

    neighbor_rows = UAM[top_neighbors, :]
    neighbor_coo = neighbor_rows.tocoo()
    recommended_artists_idx = np.unique(neighbor_coo.col)

    seed_artists = UAM.getrow(u).indices
    recommended_artists_idx = np.setdiff1d(recommended_artists_idx, seed_artists)

    if recommended_artists_idx.size > 0:
        num_to_select = min(5, recommended_artists_idx.size)
        random_elements = random.sample(list(recommended_artists_idx), num_to_select)
        rec_artist_names = [artist_ids[i] for i in random_elements]
        return (user_ids[u], recommended_user_ids, rec_artist_names)
    else:
        return (user_ids[u], recommended_user_ids, None)

# --- Main Program ---
def main():
    start_time = time.time()

    all_results = []

    # Use ThreadPoolExecutor to process all users in parallel
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [executor.submit(process_single_user, u) for u in range(num_users)]
        for future in as_completed(futures):
            res = future.result()
            all_results.append(res)

    # Print all results
    for res in all_results:
        seed_user, neighbors, rec_artists = res
        print("Seed user-id:", seed_user)
        if neighbors is None:
            print("No similar users found for", seed_user)
            print('-' * 80)
            continue
        print(f"Nearest K={K} neighbors' user-ids:", neighbors.flatten())
        if rec_artists is not None:
            print("Recommended artist names:", rec_artist_names)
        else:
            print("No new recommended artists found.")
        print('-' * 80)

    end_time = time.time()
    avg_time = (end_time - start_time) / num_users
    print(f"Average time per user recommendation: {avg_time:.4f} seconds")

if __name__ == '__main__':
    main()

Seed user-id: user_000002
Nearest K=5 neighbors' user-ids: ['user_000673' 'user_000238' 'user_000143' 'user_000726' 'user_000513']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000008
Nearest K=5 neighbors' user-ids: ['user_000864' 'user_000419' 'user_000737' 'user_000187' 'user_000123']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000007
Nearest K=5 neighbors' user-ids: ['user_000765' 'user_000236' 'user_000402' 'user_000781' 'user_000437']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kol

# cprofile

In [7]:
%prun -s cumulative main()

Seed user-id: user_000003
Nearest K=5 neighbors' user-ids: ['user_000813' 'user_000558' 'user_000400' 'user_000741' 'user_000392']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000005
Nearest K=5 neighbors' user-ids: ['user_000307' 'user_000584' 'user_000560' 'user_000807' 'user_000627']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kollektief')]
--------------------------------------------------------------------------------
Seed user-id: user_000001
Nearest K=5 neighbors' user-ids: ['user_000074' 'user_000629' 'user_000862' 'user_000844' 'user_000168']
Recommended artist names: [np.str_('Lara Martelli'), np.str_('Pina'), np.str_('Amon Düül Ii'), np.str_('Dynamoe'), np.str_('Kammerflimmer Kol

         497820 function calls (492860 primitive calls) in 1.120 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.120    1.120 {built-in method builtins.exec}
        1    0.001    0.001    1.120    1.120 <string>:1(<module>)
        1    0.002    0.002    1.119    1.119 2635966931.py:65(main)
      993    0.003    0.000    0.914    0.001 _base.py:199(as_completed)
     1774    0.003    0.000    0.909    0.001 threading.py:295(wait)
      782    0.001    0.000    0.909    0.001 threading.py:611(wait)
     7105    0.904    0.000    0.904    0.000 {method 'acquire' of '_thread.lock' objects}
     3969    0.005    0.000    0.189    0.000 {built-in method builtins.print}
    13890    0.089    0.000    0.153    0.000 interactiveshell.py:3025(write)
    13890    0.027    0.000    0.040    0.000 iostream.py:655(write)
      992    0.001    0.000    0.030    0.000 arrayprint.py:1685(_array_str_impl

# Applying Cython

In [11]:
%load_ext cython

In [12]:

%%cython
import numpy as np
cimport numpy as np

def build_playcounts_cy(str filepath):
    cdef dict counts = {}
    cdef bytes line
    with open(filepath, 'rb') as f:
        for line in f:
            parts = line.rstrip(b'\n').split(b'\t')
            counts[(parts[0], parts[3])] = counts.get((parts[0], parts[3]), 0) + 1
    return counts

def fill_csr_data_cy(dict counts,
                     np.ndarray[np.int32_t, ndim=1] data,
                     np.ndarray[np.int32_t, ndim=1] rows,
                     np.ndarray[np.int32_t, ndim=1] cols,
                     dict user_to_idx,
                     dict artist_to_idx):
    cdef Py_ssize_t i = 0
    cdef tuple key
    cdef int cnt
    for key, cnt in counts.items():
        data[i] = cnt
        rows[i] = user_to_idx[key[0]]
        cols[i] = artist_to_idx[key[1]]
        i += 1


Content of stderr:
In file included from /Users/yujinkim/.cache/ipython/cython/_cython_magic_d8bf18963cbfa55fc94ffc689c3f52d777066b59.c:1250:
In file included from /Users/yujinkim/Desktop/Efficient MRS/efficient_mrs/.venv/lib/python3.11/site-packages/numpy/_core/include/numpy/arrayobject.h:5:
In file included from /Users/yujinkim/Desktop/Efficient MRS/efficient_mrs/.venv/lib/python3.11/site-packages/numpy/_core/include/numpy/ndarrayobject.h:12:
In file included from /Users/yujinkim/Desktop/Efficient MRS/efficient_mrs/.venv/lib/python3.11/site-packages/numpy/_core/include/numpy/ndarraytypes.h:1913:
/Users/yujinkim/Desktop/Efficient MRS/efficient_mrs/.venv/lib/python3.11/site-packages/numpy/_core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^
/Users/yujinkim/.cache/ipython/cython/_cyth

In [14]:

counts = build_playcounts_cy(file_path)        # Cython: replace pandas groupby

# Create mappings: user/artist → index
user_id_to_idx     = {u:i for i,u in enumerate({k[0] for k in counts.keys()})}
artist_name_to_idx = {a:i for i,a in enumerate({k[1] for k in counts.keys()})}

# Build CSR arrays with Cython
n_nnz    = len(counts)
data_arr = np.empty(n_nnz, dtype=np.int32)
row_arr  = np.empty(n_nnz, dtype=np.int32)
col_arr  = np.empty(n_nnz, dtype=np.int32)

fill_csr_data_cy(
    counts,
    data_arr, row_arr, col_arr,
    user_id_to_idx, artist_name_to_idx
)                                              # Cython: replace Python loop

# Assemble sparse matrix exactly like before
UAM = csr_matrix(
    (data_arr, (row_arr, col_arr)),
    shape=(len(user_id_to_idx), len(artist_name_to_idx))
)

UAM = normalize(UAM, norm="l2", axis=1)

user_ids   = np.array(list(user_id_to_idx.keys()))
artist_ids = np.array(list(artist_name_to_idx.keys()))

UAM_T      = UAM.transpose().tocsr()
num_users  = UAM.shape[0]

# — Define function to process a single user —
def process_single_user(u):
    pc_vec    = UAM.getrow(u)
    sims      = pc_vec.dot(UAM_T)
    sims_coo  = sims.tocoo()

    mask             = sims_coo.col != u
    neighbor_indices = sims_coo.col[mask]
    neighbor_scores  = sims_coo.data[mask]

    if neighbor_scores.size == 0:
        return (user_ids[u], None, None)

    if neighbor_scores.size > K:
        top_k_idx    = np.argpartition(neighbor_scores, -K)[-K:]
        sorted_order = np.argsort(neighbor_scores[top_k_idx])[::-1]
        top_neighbors = neighbor_indices[top_k_idx][sorted_order]
    else:
        sorted_order  = np.argsort(neighbor_scores)[::-1]
        top_neighbors = neighbor_indices[sorted_order]

    recommended_user_ids = user_ids[top_neighbors]
    neighbor_rows       = UAM[top_neighbors, :]
    neighbor_coo        = neighbor_rows.tocoo()
    recommended_artists_idx = np.unique(neighbor_coo.col)

    seed_artists = UAM.getrow(u).indices
    recommended_artists_idx = np.setdiff1d(
        recommended_artists_idx, seed_artists
    )

    if recommended_artists_idx.size > 0:
        num_to_select   = min(5, recommended_artists_idx.size)
        random_elements = random.sample(
            list(recommended_artists_idx), num_to_select
        )
        rec_artist_names = [artist_ids[i] for i in random_elements]
        return (user_ids[u], recommended_user_ids, rec_artist_names)
    else:
        return (user_ids[u], recommended_user_ids, None)

# — Main Program —
if __name__ == "__main__":
    start_time  = time.time()
    all_results = []

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(process_single_user, u)
            for u in range(num_users)
        ]
        for future in as_completed(futures):
            res = future.result()
            all_results.append(res)

    for res in all_results:
        seed_user, neighbors, rec_artists = res
        print("Seed user-id:", seed_user)
        if neighbors is None:
            print("No similar users found for", seed_user)
            print('-' * 80)
            continue
        print(f"Nearest K={K} neighbors' user-ids:", neighbors.flatten())
        if rec_artists is not None:
            print("Recommended artist names:", rec_artists)
        else:
            print("No new recommended artists found.")
        print('-' * 80)

    end_time = time.time()
    avg_time  = (end_time - start_time) / num_users
    print(f"Average time per user recommendation: {avg_time:.4f} seconds")


Seed user-id: b'user_000526'
Nearest K=5 neighbors' user-ids: [b'user_000084' b'user_000518' b'user_000155' b'user_000285'
 b'user_000771']
Recommended artist names: [np.bytes_(b'Megamix'), np.bytes_(b'Stereophonics'), np.bytes_(b'Orgy'), np.bytes_(b'Carolina Liar'), np.bytes_(b'David Tavare')]
--------------------------------------------------------------------------------
Seed user-id: b'user_000367'
Nearest K=5 neighbors' user-ids: [b'user_000072' b'user_000018' b'user_000668' b'user_000386'
 b'user_000414']
Recommended artist names: [np.bytes_(b'The Feeling'), np.bytes_(b'Bogdan Raczynski'), np.bytes_(b'Macy Gray'), np.bytes_(b'Vanoni Paoli'), np.bytes_(b'Eagle-Eye Cherry')]
--------------------------------------------------------------------------------
Seed user-id: b'user_000406'
Nearest K=5 neighbors' user-ids: [b'user_000767' b'user_000070' b'user_000600' b'user_000219'
 b'user_000922']
Recommended artist names: [np.bytes_(b'Optiganally Yours'), np.bytes_(b'A Flock Of Seagulls